In [1]:
#Kraabime veebilehelt PDFid ja slaidiesitlused maha

import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
#Tegin siia meetodi, mis võtab nasdaqbalticu lehelt siis ISIN koodi ja kuupäeva järgi kõik .pdf formaadis failid,
#laeb need lokaalselt mulle alla.
def fetch_pdfs(ISIN,date):
    url = f"https://nasdaqbaltic.com/statistics/en/instrument/{ISIN}/reports?date={date}"
    #If there is no such folder, the script will create one automatically
    folder_location = r'C:\Users\marek.keskull\webscraping'
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    tr = soup.find("tbody")
    pdf2020 = tr.find("tr")
    
    for link in pdf2020.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)
fetch_pdfs("EE3100007857","2021-01-11")

In [2]:
#kasutan tika libraryt, selle peab panema lokaalselt käima eraldi, et kasutada tika rest teenust.
import tika
from tika import parser

import os

folder_location = r'C:\Users\marek.keskull\webscraping'
#Võtan kõik allalaetud PDF'id ning muudan need .txt failideks, et mul jääks alles ainult plain text
def extract_text_from_pdfs_recursively(dir):
    for root, dirs, files in os.walk(dir):
        for file in files:
            path_to_pdf = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_pdf)
            if ext == '.pdf':
                print("Processing " + path_to_pdf)
                pdf_contents = parser.from_file(path_to_pdf)
                path_to_txt = stem + '.txt'
                with open(path_to_txt, 'w',encoding="utf-8") as txt_file:
                    print("Writing contents to " + path_to_txt)
                    txt_file.write(pdf_contents['content'])


if __name__ == "__main__":
    extract_text_from_pdfs_recursively(folder_location)

Processing C:\Users\marek.keskull\webscraping\2020_q1_en_eur_00_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q1_en_eur_00_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q1_en_eur_con_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q1_en_eur_con_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q2_en_eur_00_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q2_en_eur_00_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q2_en_eur_con_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q2_en_eur_con_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q3_en_eur_00_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q3_en_eur_00_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q3_en_eur_con_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q3_en_eur_con_00.txt


In [32]:
#Laen teksti välja, alustan ühe failiga.
file_name = r"C:\Users\marek.keskull\webscraping\2020_q2_en_eur_00_00.txt"
text=''
with open(file_name,encoding="utf-8") as txt_file:
    for line in txt_file:
        # process the line
        text += line
#Et pandaga töötada, teen sellest dictionary. TO-DO: võiksin saada laadida kõik .txt failid siia dictionarysse!     

myDict = {"text":[text]}

myDict

import pandas as pd
pandaframe = pd.DataFrame.from_dict(myDict)
pandaframe

,text
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


# Andmete puhastamine


- Tekst kõik väikeste tähtedega
- Eemaldame punktid
- Eemaldame numbrid
- Eemdaldame ebavajaliku teksti, mis on kandiliste sulgude vahel


In [36]:
import re
import string
#esimene round teksti puhastamiseks
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)


In [37]:
#vaatame puhastatud teksti
data_clean = pd.DataFrame(pandaframe.text.apply(round1))
data_clean

,text
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [38]:
# 2. round puhastamisele
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [40]:
#vaatame puhastatud teksti
data_clean = pd.DataFrame(data_clean.text.apply(round2))
data_clean

,text
0,powerpoint presentationcoop pank groupunaudite...


In [41]:
from cleantext import clean

cleantxt = pd.DataFrame(data_clean.text.apply(round2))
cleantxt

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


,text
0,powerpoint presentationcoop pank groupunaudite...


In [53]:
companies = ['Coop']

cleantxt['Company'] = companies
cleantxt

,text,Company
0,powerpoint presentationcoop pank groupunaudite...,Coop


In [46]:
#moodustame document term matrix

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(cleantxt.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = cleantxt.index
data_dtm

,account,activity,additional,adequacy,allowances,allowancesloans,anydo,arising,audited,awareness,...,thousandcapital,total,totalling,trading,transaction,trend,volume,volumesloanschange,year,ühistukapitalothersshareholder
0,1,1,5,3,5,1,1,2,1,1,...,1,7,1,2,1,1,2,1,4,1


In [50]:
data = data_dtm.transpose()
data.head()

,0
account,1
activity,1
additional,5
adequacy,3
allowances,5


In [51]:
# top 30 sõna sõnastikus
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{0: [('portfolio', 21),
  ('loan', 16),
  ('share', 14),
  ('profit', 14),
  ('net', 13),
  ('deposits', 11),
  ('operating', 11),
  ('gross', 10),
  ('clients', 10),
  ('cost', 10),
  ('loans', 10),
  ('business', 9),
  ('income', 8),
  ('increased', 7),
  ('market', 7),
  ('total', 7),
  ('private', 7),
  ('impairment', 7),
  ('past', 7),
  ('capital', 6),
  ('payment', 6),
  ('ratio', 6),
  ('pank', 6),
  ('new', 6),
  ('losses', 5),
  ('loss', 5),
  ('additional', 5),
  ('allowances', 5),
  ('results', 4),
  ('mortgage', 4)]}